In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
metadata = pd.read_csv(r'C:\Users\win10\Downloads\movie recommendation\movies_metadata.csv',low_memory=False)
credits = pd.read_csv(r'C:\Users\win10\Downloads\movie recommendation\credits.csv')
keywords = pd.read_csv(r'C:\Users\win10\Downloads\movie recommendation\keywords.csv')
links = pd.read_csv(r'C:\Users\win10\Downloads\movie recommendation\links.csv')

In [3]:
metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [4]:
metadata.shape

(45466, 24)

In [5]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
# found duplicates

metadata.duplicated().sum()


17

In [7]:
metadata[metadata.duplicated()]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
1465,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,105045,tt0111613,de,Das Versprechen,"East-Berlin, 1961, shortly after the erection ...",...,1995-02-16,0.0,115.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,"A love, a hope, a wall.",The Promise,False,5.0,1.0
9165,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,5511,tt0062229,fr,Le Samouraï,Hitman Jef Costello is a perfectionist who alw...,...,1967-10-25,39481.0,105.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,There is no solitude greater than that of the ...,Le Samouraï,False,7.9,187.0
9327,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",NaN,23305,tt0295682,en,The Warrior,"In feudal India, a warrior (Khan) who renounce...",...,2001-09-23,0.0,86.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,The Warrior,False,6.3,15.0
13375,False,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",NaN,141971,tt1180333,fi,Blackout,Recovering from a nail gun shot to the head an...,...,2008-12-26,0.0,108.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,Which one is the first to return - memory or t...,Blackout,False,6.7,3.0
16764,False,NaN,0,"[{'id': 53, 'name': 'Thriller'}, {'id': 9648, ...",NaN,141971,tt1180333,fi,Blackout,Recovering from a nail gun shot to the head an...,...,2008-12-26,0.0,108.0,"[{'iso_639_1': 'fi', 'name': 'suomi'}]",Released,Which one is the first to return - memory or t...,Blackout,False,6.7,3.0
21165,False,NaN,0,"[{'id': 14, 'name': 'Fantasy'}, {'id': 18, 'na...",NaN,119916,tt0080000,en,The Tempest,"Prospero, the true Duke of Milan is now living...",...,1980-02-27,0.0,123.0,[],Released,NaN,The Tempest,False,0.0,0.0
21854,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 878, 'nam...",NaN,152795,tt1821641,en,The Congress,More than two decades after catapulting to sta...,...,2013-05-16,455815.0,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Congress,False,6.4,165.0
22151,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 27, 'nam...",http://www.daysofdarknessthemovie.com/,18440,tt0499456,en,Days of Darkness,When a comet strikes Earth and kicks up a clou...,...,2007-01-01,0.0,89.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Days of Darkness,False,5.0,5.0
23044,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,25541,tt1327820,da,Broderskab,Former Danish servicemen Lars and Jimmy are th...,...,2009-10-21,0.0,90.0,"[{'iso_639_1': 'da', 'name': 'Dansk'}]",Released,NaN,Brotherhood,False,7.1,21.0
24844,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.dealthemovie.com/,11115,tt0446676,en,Deal,As an ex-gambler teaches a hot-shot college ki...,...,2008-01-29,0.0,85.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Deal,False,5.2,22.0


In [8]:
metadata[metadata['id']=='23305']

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
8068,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",NaN,23305,tt0295682,en,The Warrior,"In feudal India, a warrior (Khan) who renounce...",...,2001-09-23,0.0,86.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,The Warrior,False,6.3,15.0
9327,False,NaN,0,"[{'id': 12, 'name': 'Adventure'}, {'id': 16, '...",NaN,23305,tt0295682,en,The Warrior,"In feudal India, a warrior (Khan) who renounce...",...,2001-09-23,0.0,86.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,The Warrior,False,6.3,15.0


In [9]:
idx = metadata[metadata['id'].duplicated()].index
metadata.drop(index = idx,inplace = True)



In [10]:
metadata.isnull().sum()

adult                        0
belongs_to_collection    40945
budget                       0
genres                       0
homepage                 37659
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25035
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [11]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [12]:
credits.duplicated().sum()

37

In [13]:
idx = credits[credits['id'].duplicated()].index
credits.drop(index = idx,inplace = True)



In [14]:
keywords

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [15]:
keywords.duplicated().sum()

987

In [16]:
# removing duplicates
idx = keywords[keywords['id'].duplicated()].index
keywords.drop(index = idx,inplace = True)



In [17]:
#pd.merge(metadata,credits)

# ValueError: You are trying to merge on object and int64 columns for key 'id'. If you wish to proceed you should use pd.concat

In [18]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45432 non-null  object
 1   crew    45432 non-null  object
 2   id      45432 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [46]:
# getting error while merging due to dif data type / so now changing

# metadata['id'] = metadata['id'].astype(int)


# ValueError: invalid literal for int() with base 10: '1997-08-20'

In [48]:
## there were few dates mentioned in id column, to find them

metadata[metadata['id'].apply(lambda x:x.isdigit())==False]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:

metadata.drop(index = [19730,29503,35587],inplace = True)

In [52]:
metadata['id'] = metadata['id'].astype(int)

In [54]:

df = pd.merge(metadata,credits,on="id")

In [56]:
df = pd.merge(df,keywords,on="id")

In [58]:
df = pd.merge(df,links,left_on="id",right_on = 'movieId')

In [60]:
df.shape

(7635, 30)

In [62]:
df = df[:2400] # reducing the data to deploy


In [64]:
pd.set_option('display.max_columns', None)
df.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords,movieId,imdbId,tmdbId
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",862,116985,88224.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",8844,78763,42164.0


In [66]:
df = df[['id','original_title','belongs_to_collection','genres','overview','production_companies','tagline'
    ,'cast','crew','keywords','tmdbId']]

In [68]:
df.head(2)

,id,original_title,belongs_to_collection,genres,overview,production_companies,tagline,cast,crew,keywords,tmdbId
0,862,Toy Story,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]",NaN,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...",88224.0
1,8844,Jumanji,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...",Roll the dice and unleash the excitement!,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...","[{'id': 10090, 'name': 'board game'}, {'id': 1...",42164.0


In [70]:
df['belongs_to_collection']  #

0       {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                     NaN
2                                                     NaN
3       {'id': 645, 'name': 'James Bond Collection', '...
4                                                     NaN
                              ...                        
2395                                                  NaN
2396                                                  NaN
2397                                                  NaN
2398    {'id': 96937, 'name': 'The Boogeyman Collectio...
2399                                                  NaN
Name: belongs_to_collection, Length: 2400, dtype: object

In [72]:
type(df['belongs_to_collection'][0])

str

In [74]:
def converter(col):
    l = []
    if pd.isna(col):
        pass
    else: l.append(eval(col)['name'])
    return l
    

In [76]:
df['belongs_to_collection'] = df['belongs_to_collection'].apply(converter)

In [78]:
def converter2(col):
    l = []
    for i in eval(col):
        l.append(i['name'])
    return l

In [80]:
df['genres'] = df['genres'].apply(converter2)

In [82]:
df['production_companies'] = df['production_companies'].apply(converter2)

In [84]:
def converter1(col):
    l = []
    if pd.isna(col):
        pass
    else: l = col.split()
    return l

In [86]:
df['tagline'] = df['tagline'].apply(converter1)

In [88]:
df['overview'] = df['overview'].apply(converter1)

In [90]:
df['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [92]:
def converter3(col):
    l = []
    counter = 0
    for i in eval(col):
        if counter <= 3: 
            l.append(i['name'])
            counter = counter + 1
    return l

In [94]:
df['cast'] = df['cast'].apply(converter3)

In [95]:
def converter4(col):
    l = []
   
    for i in eval(col):
        if i['job']=='Director': 
            l.append(i['name'])
    return l

In [96]:
df['crew'] = df['crew'].apply(converter4)

In [99]:
df['keywords'] = df['keywords'].apply(converter2)

In [100]:
df['tmdbId'].fillna(0,inplace = True) # converting NaN values to 0 to convert it to int from float

C:\Users\win10\AppData\Local\Temp\ipykernel_9900\2803442578.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['tmdbId'].fillna(0,inplace = True) # converting NaN values to 0 to convert it to int from float


In [110]:
df['tmdbId'] = df['tmdbId'].astype(int)

In [112]:
df.head(2)

,id,original_title,belongs_to_collection,genres,overview,production_companies,tagline,cast,crew,keywords,tmdbId,tags
0,862,Toy Story,[Toy Story Collection],"[Animation, Comedy, Family]","[Led, by, Woody,, Andy's, toys, live, happily,...",[Pixar Animation Studios],[],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney]",[John Lasseter],"[jealousy, toy, boy, friendship, friends, riva...",88224,"[Toy Story Collection, Animation, Comedy, Fami..."
1,8844,Jumanji,[],"[Adventure, Fantasy, Family]","[When, siblings, Judy, and, Peter, discover, a...","[TriStar Pictures, Teitler Film, Interscope Co...","[Roll, the, dice, and, unleash, the, excitement!]","[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",[Joe Johnston],"[board game, disappearance, based on children'...",42164,"[Adventure, Fantasy, Family, When, siblings, J..."


In [114]:
# merging all text columns together

df['tags'] = df['belongs_to_collection'] + df['genres'] + df['overview'] + df['production_companies'] + df['tagline'] + df['cast'] + df['crew'] + df['keywords']

In [116]:
df['tags'][0]

['Toy Story Collection',
 'Animation',
 'Comedy',
 'Family',
 'Led',
 'by',
 'Woody,',
 "Andy's",
 'toys',
 'live',
 'happily',
 'in',
 'his',
 'room',
 'until',
 "Andy's",
 'birthday',
 'brings',
 'Buzz',
 'Lightyear',
 'onto',
 'the',
 'scene.',
 'Afraid',
 'of',
 'losing',
 'his',
 'place',
 'in',
 "Andy's",
 'heart,',
 'Woody',
 'plots',
 'against',
 'Buzz.',
 'But',
 'when',
 'circumstances',
 'separate',
 'Buzz',
 'and',
 'Woody',
 'from',
 'their',
 'owner,',
 'the',
 'duo',
 'eventually',
 'learns',
 'to',
 'put',
 'aside',
 'their',
 'differences.',
 'Pixar Animation Studios',
 'Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'Jim Varney',
 'John Lasseter',
 'jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy next door',
 'new toy',
 'toy comes to life']

In [118]:
# removing space from each charactor

df['tags'] = df['tags'].apply(lambda x:[i.replace(" ","") for i in x])

In [120]:
df.drop(['belongs_to_collection','genres','overview','production_companies','tagline','cast','crew','keywords'],axis = 1,inplace = True)

In [122]:
df.head(3)

,id,original_title,tmdbId,tags
0,862,Toy Story,88224,"[ToyStoryCollection, Animation, Comedy, Family..."
1,8844,Jumanji,42164,"[Adventure, Fantasy, Family, When, siblings, J..."
2,949,Heat,220,"[Action, Crime, Drama, Thriller, Obsessive, ma..."


In [124]:
# converting list to string

df['tags'] = df['tags'].apply(lambda x:" ".join(x))

In [126]:
df.head(3)

,id,original_title,tmdbId,tags
0,862,Toy Story,88224,ToyStoryCollection Animation Comedy Family Led...
1,8844,Jumanji,42164,Adventure Fantasy Family When siblings Judy an...
2,949,Heat,220,Action Crime Drama Thriller Obsessive master t...


In [128]:
ps = PorterStemmer()  # bring all words to same ground

In [130]:
def stemming(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))
    return " ".join(l)    

In [132]:
df['tags'] = df['tags'].apply(stemming)

In [133]:
# vectorizing

tfidf = TfidfVectorizer(max_features = 2400, stop_words = 'english')

In [134]:
vector = tfidf.fit_transform(df['tags']).toarray()

In [135]:
vector.shape

(2400, 2400)

In [136]:
tfidf.get_feature_names_out()[:1000]  

array(['000', '007', '10', '11', '12', '15', '1930', '1940', '1950',
       '1960', '1970', '1980', '19th', '20', '20th', '30', '60',
       'abandon', 'abil', 'abl', 'aboard', 'abus', 'academi', 'accept',
       'access', 'accid', 'accident', 'accompani', 'account', 'accus',
       'achiev', 'acquir', 'act', 'action', 'actor', 'actress', 'actual',
       'adam', 'adapt', 'addict', 'admir', 'adolesc', 'adopt', 'adult',
       'adulteri', 'advanc', 'adventur', 'adventure', 'advertis',
       'affair', 'affect', 'afraid', 'africa', 'african',
       'aftercreditssting', 'afterlif', 'ag', 'age', 'agediffer', 'agent',
       'ago', 'agre', 'aid', 'air', 'airplan', 'airport', 'albert',
       'alcohol', 'alecbaldwin', 'alex', 'alfredhitchcock', 'alic',
       'alien', 'aliv', 'alive', 'alli', 'allow', 'alon', 'alpacino',
       'alphavillefilm', 'alreadi', 'alter', 'altern', 'alway', 'ambiti',
       'amblinentertain', 'ambush', 'america', 'american',
       'americanzoetrop', 'amnesia', 'a

In [137]:
cos_sim = cosine_similarity(vector)

In [138]:
cos_sim.shape

(2400, 2400)

In [146]:
cos_sim[0]

array([1.        , 0.05657673, 0.        , ..., 0.        , 0.0414869 ,
       0.00481515])

In [148]:
def recommendation(movie):
    indx = df[df['original_title']==movie].index[0]  # 
    movie_list = sorted(list(enumerate(cos_sim[indx])),reverse=True,key = lambda x:x[1])[1:6] # getting top 5 movies with highest similarity score

    for i in movie_list:
        print(df.iloc[i[0]].original_title)
    


In [150]:
##
def recom(movie):
    indx = df[df['original_title']==movie].index[0]  # 
    vec = vector[indx].reshape(1,-1)
    similarity = cosine_similarity(vec,vector)
    movie_list = sorted(list(enumerate(similarity.flatten())),reverse=True,key = lambda x:x[1])[1:6] # getting top 5 movies with highest similarity score

    for i in movie_list:
        print(df.iloc[i[0]].original_title)

In [152]:
df[df['original_title']=='Batman'].index[0]

159

In [154]:
recommendation('Batman')

Batman & Robin
Batman Returns
Superman
Batman Forever
Superman II


In [156]:
df.iloc[863]

id                                                            37853
original_title                                       Mister Roberts
tmdbId                                                        11968
tags              misterrobertscollect comedi drama famili war a...
Name: 863, dtype: object

In [158]:
import pickle

In [160]:
pickle.dump(df,open('movies.pkl','wb'))

In [162]:
# pickle.dump(cos_sim,open('cos_sim.pkl','wb'))

In [164]:
import joblib
joblib.dump(cos_sim, "cosi_simi.pkl", compress=3)

['cosi_simi.pkl']

In [165]:
joblib.dump(vector, "vector.pkl", compress=3)

['vector.pkl']